In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
def get_anchors(url="https://atlas.ripe.net/api/v2/anchors/?page=1"):
    response = requests.get(url)
    if response.status_code == 200:
        anchors = response.json()["results"]
        next_url = response.json()["next"]
    else:
        print("Failed to retrieve anchors. Status code:", response.status_code)
    return anchors, next_url


def get_anchor_measurements(url="https://atlas.ripe.net/api/v2/anchor-measurements/?page=2"):
    response = requests.get(url)
    if response.status_code == 200:
        msrmnt = response.json()["results"]
        next_url = response.json()["next"]
    else:
        print("Failed to retrieve anchor measurements. Status code:", response.status_code)
    return msrmnt, next_url


def get_am_results(url):
    #src, dest, rtt
    response = requests.get(url + "latest")
    out = []
    if response.status_code == 200:
        results = response.json()
        for ele in results:
            rtt = 0
            cnt = 0
            for tmp in ele["result"]:
                if "rtt" in tmp:
                    rtt = rtt + tmp["rtt"]
                    cnt = cnt + 1
            if cnt > 0:
                rtt = rtt / cnt
            #sub_result = {"src": ele["src_addr"], "dst": ele["dst_addr"], "rtt": rtt}
            sub_result_str = ele["src_addr"] + "%%" + ele["dst_addr"] + "%%" + str(int(rtt))
            out.append(sub_result_str)
    else:
        print("Failed to retrieve anchor measurements. Status code:", response.status_code)
    return out

def get_and_write_rtt(anchor_measurements, start, offset, filename, checkpoint):
    rtt_list = []
    if (start + offset) >= len(anchor_measurements):
        rmax = len(anchor_measurements)
    else:
        rmax = start + offset
    for i in range(start, rmax):
        am = anchor_measurements[i]
        if am["type"] == "ping":
            #anchor_measurements_latest[am["target"]] = get_am_results(am["measurement"])
            rtt_list.extend(get_am_results(am["measurement"]))
            #print(i, " ", am["target"])
        if i > 0 and i != start and i % checkpoint == 0:
            print("Checkpoint ", i)
            with open(filename, "a") as outfile:
                outfile.write("\n".join(rtt_list))
                outfile.write("\n")
            rtt_list = []

    print("Finished! Saving remaining")
    with open(filename, "a") as outfile:
        outfile.write("\n".join(rtt_list))
    print("Done")

def get_all(func):
    page_cnt = 1
    result = []
    sub_result, next_url = func()
    result.extend(sub_result)

    while next_url:
        page_cnt = page_cnt + 1
        print(page_cnt, " ", next_url)
        sub_result, next_url = func(next_url)
        result.extend(sub_result)
    return result

In [3]:
anchors = get_all(get_anchors)
print(len(anchors))

2   https://atlas.ripe.net/api/v2/anchors/?page=2
3   https://atlas.ripe.net/api/v2/anchors/?page=3
4   https://atlas.ripe.net/api/v2/anchors/?page=4
5   https://atlas.ripe.net/api/v2/anchors/?page=5
6   https://atlas.ripe.net/api/v2/anchors/?page=6
7   https://atlas.ripe.net/api/v2/anchors/?page=7
8   https://atlas.ripe.net/api/v2/anchors/?page=8
9   https://atlas.ripe.net/api/v2/anchors/?page=9
10   https://atlas.ripe.net/api/v2/anchors/?page=10
11   https://atlas.ripe.net/api/v2/anchors/?page=11
12   https://atlas.ripe.net/api/v2/anchors/?page=12
13   https://atlas.ripe.net/api/v2/anchors/?page=13
1230


In [4]:
anchor_measurements = get_all(get_anchor_measurements)

2   https://atlas.ripe.net/api/v2/anchor-measurements/?page=3
3   https://atlas.ripe.net/api/v2/anchor-measurements/?page=4
4   https://atlas.ripe.net/api/v2/anchor-measurements/?page=5
5   https://atlas.ripe.net/api/v2/anchor-measurements/?page=6
6   https://atlas.ripe.net/api/v2/anchor-measurements/?page=7
7   https://atlas.ripe.net/api/v2/anchor-measurements/?page=8
8   https://atlas.ripe.net/api/v2/anchor-measurements/?page=9
9   https://atlas.ripe.net/api/v2/anchor-measurements/?page=10
10   https://atlas.ripe.net/api/v2/anchor-measurements/?page=11
11   https://atlas.ripe.net/api/v2/anchor-measurements/?page=12
12   https://atlas.ripe.net/api/v2/anchor-measurements/?page=13
13   https://atlas.ripe.net/api/v2/anchor-measurements/?page=14
14   https://atlas.ripe.net/api/v2/anchor-measurements/?page=15
15   https://atlas.ripe.net/api/v2/anchor-measurements/?page=16
16   https://atlas.ripe.net/api/v2/anchor-measurements/?page=17
17   https://atlas.ripe.net/api/v2/anchor-measurements/

In [5]:
anchor_measurements

[{'id': 397,
  'is_mesh': True,
  'measurement': 'https://atlas.ripe.net/api/v2/measurements/1817189/',
  'target': 'https://atlas.ripe.net/api/v2/anchors/490/',
  'type': 'traceroute'},
 {'id': 398,
  'is_mesh': True,
  'measurement': 'https://atlas.ripe.net/api/v2/measurements/1817190/',
  'target': 'https://atlas.ripe.net/api/v2/anchors/490/',
  'type': 'ping'},
 {'id': 399,
  'is_mesh': True,
  'measurement': 'https://atlas.ripe.net/api/v2/measurements/1817191/',
  'target': 'https://atlas.ripe.net/api/v2/anchors/490/',
  'type': 'traceroute'},
 {'id': 400,
  'is_mesh': True,
  'measurement': 'https://atlas.ripe.net/api/v2/measurements/1817192/',
  'target': 'https://atlas.ripe.net/api/v2/anchors/490/',
  'type': 'ping'},
 {'id': 417,
  'is_mesh': True,
  'measurement': 'https://atlas.ripe.net/api/v2/measurements/1845531/',
  'target': 'https://atlas.ripe.net/api/v2/anchors/527/',
  'type': 'traceroute'},
 {'id': 418,
  'is_mesh': True,
  'measurement': 'https://atlas.ripe.net/api/

In [6]:
len(anchor_measurements)

11398

In [15]:
start = 0
#offset = 100
offset = len(anchor_measurements)
filename = "datasets/RIPEAtlas/anchor_rtt.csv"
checkpoint = 100

get_and_write_rtt(anchor_measurements, start, offset, filename, checkpoint)

Checkpoint  100
Checkpoint  200
Checkpoint  300
Checkpoint  400
Checkpoint  500
Checkpoint  600
Checkpoint  700
Checkpoint  800
Checkpoint  900
Checkpoint  1000
Checkpoint  1100
Checkpoint  1200
Checkpoint  1300
Checkpoint  1400
Checkpoint  1500
Checkpoint  1600
Checkpoint  1700
Checkpoint  1800
Checkpoint  1900
Checkpoint  2000
Checkpoint  2100
Checkpoint  2200
Checkpoint  2300
Checkpoint  2400
Checkpoint  2500
Checkpoint  2600
Checkpoint  2700
Checkpoint  2800
Checkpoint  2900
Checkpoint  3000
Checkpoint  3100
Checkpoint  3200
Checkpoint  3300
Checkpoint  3400
Checkpoint  3500
Checkpoint  3600
Checkpoint  3700
Checkpoint  3800
Checkpoint  3900
Checkpoint  4000
Checkpoint  4100
Checkpoint  4200
Checkpoint  4300
Checkpoint  4400
Checkpoint  4500
Checkpoint  4600
Checkpoint  4700
Checkpoint  4800
Checkpoint  4900
Checkpoint  5000
Checkpoint  5100
Checkpoint  5200
Checkpoint  5300
Checkpoint  5400
Checkpoint  5500
Checkpoint  5600
Checkpoint  5700
Checkpoint  5800
Checkpoint  5900
Checkp

In [4]:
anchors

[{'id': 2182,
  'type': 'Anchor',
  'fqdn': 'ae-auh-as208800.anchors.atlas.ripe.net',
  'probe': 6771,
  'is_ipv4_only': True,
  'ip_v4': '91.201.7.243',
  'as_v4': 208800,
  'ip_v4_gateway': None,
  'ip_v4_netmask': None,
  'ip_v6': None,
  'as_v6': None,
  'ip_v6_gateway': None,
  'ip_v6_prefix': None,
  'city': 'Abu Dhabi',
  'country': 'AE',
  'company': 'EOS CLOUD TECHNOLOGY L.L.C.',
  'nic_handle': '',
  'geometry': {'type': 'Point', 'coordinates': [54.3773438, 24.453884]},
  'tlsa_record': '',
  'is_disabled': False,
  'date_live': '2020-04-16T14:20:00.410044',
  'hardware_version': 99},
 {'id': 2825,
  'type': 'Anchor',
  'fqdn': 'ae-dxb-as15412-client.anchors.atlas.ripe.net',
  'probe': 7047,
  'is_ipv4_only': False,
  'ip_v4': '80.77.4.60',
  'as_v4': 15412,
  'ip_v4_gateway': None,
  'ip_v4_netmask': None,
  'ip_v6': '2001:1a00:acca:130d::4',
  'as_v6': 15412,
  'ip_v6_gateway': None,
  'ip_v6_prefix': None,
  'city': 'Dubai',
  'country': 'AE',
  'company': 'Emirates Integr

In [5]:
ip_dict = {}
anchor_ids = set()
for a in anchors:
    ipv4 = a["ip_v4"]
    ipv6 = a["ip_v6"]
    id = a["id"]
    ip_dict[ipv4] = id
    ip_dict[ipv6] = id
    anchor_ids.add(id)

In [6]:
ip_dict

{'91.201.7.243': 2182,
 None: 1000,
 '80.77.4.60': 2825,
 '2001:1a00:acca:130d::4': 2825,
 '94.203.76.219': 2581,
 '2a00:f28:f::219': 2581,
 '87.201.194.242': 1421,
 '2a00:f28:300:1::2': 1421,
 '63.222.248.5': 3387,
 '2400:8800:e011::5': 3387,
 '185.34.2.114': 1457,
 '2a00:dd80:3f:50::2': 1457,
 '37.252.245.94': 1213,
 '2a00:11c0:14:aa3::a': 1213,
 '94.207.43.2': 2582,
 '2a00:f2f:1106::ffff:5ecf:2b02': 2582,
 '185.92.128.30': 2431,
 '2a04:2600:1:1::30': 2431,
 '156.154.53.254': 2471,
 '2610:a1:3024:252::254': 2471,
 '194.1.149.210': 1029,
 '2001:7f9:4::10': 1029,
 '37.139.119.110': 1205,
 '91.218.5.67': 1765,
 '2001:7f8:94:3::a': 1765,
 '190.124.192.162': 3066,
 '2801:0:90:25::162': 3066,
 '170.210.5.200': 669,
 '2800:110:5::200': 669,
 '200.61.6.78': 1936,
 '143.202.20.39': 1933,
 '2803:4e40:0:18:a71a:ffff:0:cccc': 1933,
 '138.204.7.245': 1938,
 '2803:2240:1:5:0:f:0:c': 1938,
 '200.3.170.54': 3148,
 '2001:13e8:e::54': 3148,
 '94.199.170.201': 1370,
 '2a02:1688::401': 1370,
 '80.92.124

In [7]:
len(anchor_ids)

1230

In [8]:
anchor_ids = list(anchor_ids)
df = pd.DataFrame(np.nan, index=anchor_ids, columns=anchor_ids)

In [9]:
df

,1,3,4,5,6,7,8,10,11,12,...,3447,3448,3449,3450,3452,3453,3455,3460,3471,3478
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
filename = "datasets/RIPEAtlas/19062023/anchor_rtt.csv"
not_found = 0

def fill_df(filename, df):
    not_found = 0
    ips = set(df.columns)
    with open(filename, 'r') as fin:
        lines = fin.readlines()
        for line in lines:
            sl = line.split("%%")
            if len(sl) == 3 and sl[0] in ips and sl[1] in ips:
                df.at[sl[0], sl[1]] = int(sl[2])
                df.at[sl[1], sl[0]] = int(sl[2])
            else:
                not_found = not_found + 1
    print("Number of invalid elements is ", not_found)
    return df


def get_ips(filename):
    out = set()
    invalids = 0
    with open(filename, 'r') as fin:
        lines = fin.readlines()
        for line in lines:
            sl = line.split("%%")
            if len(sl) == 3 and len(sl[0]) > 1 and len(sl[1]) > 1:
                src_id = sl[0]
                dst_id = sl[1]
                out.add(src_id)
                out.add(dst_id)
            else:
                invalids = invalids + 1
                #print("Invalid line ", line)
    print("Number of invalids is ", invalids)
    return out

In [56]:
ip_list = list(get_ips(filename))
ip_list.sort()

Number of invalids is  854


In [57]:
print(ip_list)
len(ip_list)

['10.0.0.1', '10.0.0.10', '10.0.0.100', '10.0.0.101', '10.0.0.102', '10.0.0.104', '10.0.0.106', '10.0.0.107', '10.0.0.108', '10.0.0.11', '10.0.0.110', '10.0.0.112', '10.0.0.113', '10.0.0.115', '10.0.0.116', '10.0.0.117', '10.0.0.118', '10.0.0.121', '10.0.0.122', '10.0.0.123', '10.0.0.124', '10.0.0.125', '10.0.0.126', '10.0.0.128', '10.0.0.13', '10.0.0.130', '10.0.0.131', '10.0.0.132', '10.0.0.134', '10.0.0.135', '10.0.0.137', '10.0.0.138', '10.0.0.14', '10.0.0.140', '10.0.0.142', '10.0.0.143', '10.0.0.145', '10.0.0.146', '10.0.0.15', '10.0.0.153', '10.0.0.154', '10.0.0.155', '10.0.0.156', '10.0.0.158', '10.0.0.159', '10.0.0.16', '10.0.0.160', '10.0.0.161', '10.0.0.167', '10.0.0.170', '10.0.0.175', '10.0.0.176', '10.0.0.178', '10.0.0.18', '10.0.0.180', '10.0.0.181', '10.0.0.182', '10.0.0.183', '10.0.0.187', '10.0.0.188', '10.0.0.189', '10.0.0.19', '10.0.0.190', '10.0.0.191', '10.0.0.193', '10.0.0.196', '10.0.0.197', '10.0.0.198', '10.0.0.2', '10.0.0.200', '10.0.0.201', '10.0.0.202', '10

14778

In [58]:
df = pd.DataFrame(np.nan, index=ip_list, columns=ip_list)
df = fill_df(filename, df)

Number of invalid elements is  854


In [62]:
df

,10.0.0.1,10.0.0.10,10.0.0.100,10.0.0.101,10.0.0.102,10.0.0.104,10.0.0.106,10.0.0.107,10.0.0.108,10.0.0.11,...,fe80::5054:ff:fea1:e99,fe80::c24a:ff:fe09:5570,fe80::c66e:1fff:fe5b:f406,fe80::da58:d7ff:fe03:40c,fe80::da58:d7ff:fe03:741,fe80::da58:d7ff:fe03:798,fe80::da58:d7ff:fe03:b45,fe80::ea94:f6ff:fe48:611c,fe80::eade:27ff:fec9:6930,fe80::eade:27ff:fec9:77d2
10.0.0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10.0.0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10.0.0.100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10.0.0.101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10.0.0.102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fe80::da58:d7ff:fe03:798,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fe80::da58:d7ff:fe03:b45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fe80::ea94:f6ff:fe48:611c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fe80::eade:27ff:fec9:6930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
df.to_csv("datasets/RIPEAtlas/19062023/matrix_raw.csv", header=True, index=False)

In [64]:
na_thresh = 14000
df = df[df.isnull().sum(axis=1) < na_thresh]

In [65]:
df

,10.0.0.1,10.0.0.10,10.0.0.100,10.0.0.101,10.0.0.102,10.0.0.104,10.0.0.106,10.0.0.107,10.0.0.108,10.0.0.11,...,fe80::5054:ff:fea1:e99,fe80::c24a:ff:fe09:5570,fe80::c66e:1fff:fe5b:f406,fe80::da58:d7ff:fe03:40c,fe80::da58:d7ff:fe03:741,fe80::da58:d7ff:fe03:798,fe80::da58:d7ff:fe03:b45,fe80::ea94:f6ff:fe48:611c,fe80::eade:27ff:fec9:6930,fe80::eade:27ff:fec9:77d2
10.128.0.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10.132.0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10.138.0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10.140.0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10.142.0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95.86.44.5,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96.126.70.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98.159.46.220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99.79.30.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
df2 = df[list(df.index)]

In [67]:
df2

,10.128.0.17,10.132.0.2,10.138.0.2,10.140.0.2,10.142.0.3,10.146.0.2,10.148.0.2,10.148.148.108,10.150.0.3,10.152.0.2,...,94.247.111.19,94.77.252.90,95.163.252.10,95.217.19.69,95.47.236.146,95.86.44.5,96.126.70.90,98.159.46.220,99.79.30.74,fe80::5054:ff:fea1:e99
10.128.0.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,185.0,175.0,136.0,124.0,136.0,0.0,42.0,0.0,0.0,NaN
10.132.0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,91.0,78.0,43.0,33.0,39.0,0.0,99.0,0.0,0.0,NaN
10.138.0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,215.0,207.0,167.0,155.0,168.0,0.0,73.0,0.0,0.0,NaN
10.140.0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,333.0,292.0,285.0,273.0,281.0,0.0,192.0,0.0,0.0,NaN
10.142.0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,173.0,163.0,123.0,112.0,124.0,0.0,80.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95.86.44.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
96.126.70.90,42.0,99.0,73.0,192.0,80.0,162.0,225.0,96.0,31.0,210.0,...,178.0,178.0,127.0,107.0,111.0,0.0,0.0,0.0,0.0,NaN
98.159.46.220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN
99.79.30.74,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN


In [68]:
df2.isna().sum()

10.128.0.17                853
10.132.0.2                 853
10.138.0.2                 853
10.140.0.2                 853
10.142.0.3                 853
                          ... 
95.86.44.5                 894
96.126.70.90               783
98.159.46.220             1065
99.79.30.74               1012
fe80::5054:ff:fea1:e99    1025
Length: 1758, dtype: int64

In [69]:
df3 = df2[df2.isnull().sum(axis=1) < 900]
df3 = df3[list(df3.index)]
df3.isna().sum()

10.128.0.17      69
10.132.0.2       69
10.138.0.2       69
10.140.0.2       69
10.142.0.3       69
                 ..
95.163.252.10     0
95.217.19.69      0
95.47.236.146     0
95.86.44.5        1
96.126.70.90      0
Length: 793, dtype: int64

In [70]:
df4 = df3[df3.isnull().sum(axis=1) < 10]
df4 = df4[list(df4.index)]
df4.isna().sum()

101.53.31.6        0
102.130.49.144     0
102.130.68.151     0
102.219.127.20     0
102.222.106.178    0
                  ..
95.163.252.10      0
95.217.19.69       0
95.47.236.146      0
95.86.44.5         0
96.126.70.90       0
Length: 723, dtype: int64

In [71]:
df4

,101.53.31.6,102.130.49.144,102.130.68.151,102.219.127.20,102.222.106.178,103.102.166.20,103.113.176.82,103.116.125.2,103.121.34.150,103.122.190.141,...,94.231.212.79,94.24.115.154,94.242.1.50,94.247.111.19,94.77.252.90,95.163.252.10,95.217.19.69,95.47.236.146,95.86.44.5,96.126.70.90
101.53.31.6,0.0,306.0,166.0,22.0,297.0,23.0,52.0,22.0,231.0,72.0,...,245.0,274.0,196.0,256.0,232.0,289.0,273.0,212.0,0.0,235.0
102.130.49.144,306.0,0.0,0.0,196.0,40.0,115.0,236.0,211.0,339.0,322.0,...,0.0,161.0,207.0,256.0,234.0,222.0,195.0,196.0,0.0,251.0
102.130.68.151,166.0,0.0,0.0,144.0,223.0,144.0,169.0,144.0,326.0,194.0,...,0.0,160.0,210.0,260.0,256.0,215.0,198.0,223.0,272.0,254.0
102.219.127.20,22.0,196.0,144.0,0.0,317.0,0.0,26.0,1.0,296.0,50.0,...,0.0,233.0,197.0,247.0,264.0,205.0,184.0,286.0,284.0,313.0
102.222.106.178,297.0,40.0,223.0,317.0,0.0,250.0,280.0,251.0,368.0,348.0,...,301.0,240.0,251.0,307.0,225.0,250.0,237.0,235.0,0.0,289.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95.163.252.10,289.0,222.0,215.0,205.0,250.0,203.0,192.0,189.0,325.0,243.0,...,35.0,68.0,9.0,51.0,129.0,0.0,16.0,17.0,0.0,127.0
95.217.19.69,273.0,195.0,198.0,184.0,237.0,196.0,197.0,170.0,315.0,220.0,...,52.0,51.0,6.0,93.0,107.0,16.0,0.0,11.0,0.0,107.0
95.47.236.146,212.0,196.0,223.0,286.0,235.0,169.0,198.0,168.0,279.0,224.0,...,82.0,56.0,17.0,68.0,132.0,17.0,11.0,0.0,0.0,111.0
95.86.44.5,0.0,0.0,272.0,284.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
df_cleaned = df4.fillna(0)

In [75]:
df_cleaned

,101.53.31.6,102.130.49.144,102.130.68.151,102.219.127.20,102.222.106.178,103.102.166.20,103.113.176.82,103.116.125.2,103.121.34.150,103.122.190.141,...,94.231.212.79,94.24.115.154,94.242.1.50,94.247.111.19,94.77.252.90,95.163.252.10,95.217.19.69,95.47.236.146,95.86.44.5,96.126.70.90
101.53.31.6,0.0,306.0,166.0,22.0,297.0,23.0,52.0,22.0,231.0,72.0,...,245.0,274.0,196.0,256.0,232.0,289.0,273.0,212.0,0.0,235.0
102.130.49.144,306.0,0.0,0.0,196.0,40.0,115.0,236.0,211.0,339.0,322.0,...,0.0,161.0,207.0,256.0,234.0,222.0,195.0,196.0,0.0,251.0
102.130.68.151,166.0,0.0,0.0,144.0,223.0,144.0,169.0,144.0,326.0,194.0,...,0.0,160.0,210.0,260.0,256.0,215.0,198.0,223.0,272.0,254.0
102.219.127.20,22.0,196.0,144.0,0.0,317.0,0.0,26.0,1.0,296.0,50.0,...,0.0,233.0,197.0,247.0,264.0,205.0,184.0,286.0,284.0,313.0
102.222.106.178,297.0,40.0,223.0,317.0,0.0,250.0,280.0,251.0,368.0,348.0,...,301.0,240.0,251.0,307.0,225.0,250.0,237.0,235.0,0.0,289.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95.163.252.10,289.0,222.0,215.0,205.0,250.0,203.0,192.0,189.0,325.0,243.0,...,35.0,68.0,9.0,51.0,129.0,0.0,16.0,17.0,0.0,127.0
95.217.19.69,273.0,195.0,198.0,184.0,237.0,196.0,197.0,170.0,315.0,220.0,...,52.0,51.0,6.0,93.0,107.0,16.0,0.0,11.0,0.0,107.0
95.47.236.146,212.0,196.0,223.0,286.0,235.0,169.0,198.0,168.0,279.0,224.0,...,82.0,56.0,17.0,68.0,132.0,17.0,11.0,0.0,0.0,111.0
95.86.44.5,0.0,0.0,272.0,284.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [76]:
df_cleaned.to_csv("datasets/RIPEAtlas/19062023/matrix_cleaned.csv", header=True, index=False)